# Part C

In the previous part B, we get 

MSE of 42.8598274048752 and 

MSE deviation of 22.198094384171497

In part C, I am going to set epochs as 100 and see the differences.

As I mentioned in Part B, all the steps are the same, except in this part we are going to set epochs as 100.

## Get data and normalize

Before we go further we have to check whether we can use  our gpu option in our jupyter notebook. (You can skip this if you just use cpu option in jupyter notebook)

In [2]:
import tensorflow as tf
import pandas as pd
import numpy as np
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'
#this part is for gpu run in jupyter check. If you just use cpu based jupyter notebook, you can skip this.
tf.test.is_gpu_available(
    cuda_only=False, min_cuda_compute_capability=None
)

True

Since it returns true, now we are ready to move on to the next step.

Let's check our data and see whethere we can use this or not.

In [3]:
concrete_data = pd.read_csv('https://cocl.us/concrete_data')
concrete_data.head()

,Cement,Blast Furnace Slag,Fly Ash,Water,Superplasticizer,Coarse Aggregate,Fine Aggregate,Age,Strength
0,540.0,0.0,0.0,162.0,2.5,1040.0,676.0,28,79.99
1,540.0,0.0,0.0,162.0,2.5,1055.0,676.0,28,61.89
2,332.5,142.5,0.0,228.0,0.0,932.0,594.0,270,40.27
3,332.5,142.5,0.0,228.0,0.0,932.0,594.0,365,41.05
4,198.6,132.4,0.0,192.0,0.0,978.4,825.5,360,44.30


In [6]:
concrete_data.shape


(1030, 9)

In [8]:
concrete_data.describe()


,Cement,Blast Furnace Slag,Fly Ash,Water,Superplasticizer,Coarse Aggregate,Fine Aggregate,Age,Strength
count,1030.000000,1030.000000,1030.000000,1030.000000,1030.000000,1030.000000,1030.000000,1030.000000,1030.000000
mean,281.167864,73.895825,54.188350,181.567282,6.204660,972.918932,773.580485,45.662136,35.817961
std,104.506364,86.279342,63.997004,21.354219,5.973841,77.753954,80.175980,63.169912,16.705742
min,102.000000,0.000000,0.000000,121.800000,0.000000,801.000000,594.000000,1.000000,2.330000
25%,192.375000,0.000000,0.000000,164.900000,0.000000,932.000000,730.950000,7.000000,23.710000
50%,272.900000,22.000000,0.000000,185.000000,6.400000,968.000000,779.500000,28.000000,34.445000
75%,350.000000,142.950000,118.300000,192.000000,10.200000,1029.400000,824.000000,56.000000,46.135000
max,540.000000,359.400000,200.100000,247.000000,32.200000,1145.000000,992.600000,365.000000,82.600000


In [9]:
concrete_data.isnull().sum()

Cement                0
Blast Furnace Slag    0
Fly Ash               0
Water                 0
Superplasticizer      0
Coarse Aggregate      0
Fine Aggregate        0
Age                   0
Strength              0
dtype: int64

Since there are no null values, we can assume this data is safe to use.

Now, we must set predictors and target for our model.

In [10]:
concrete_data_columns = concrete_data.columns
predictors = concrete_data[concrete_data_columns[concrete_data_columns != 'Strength']] # all columns except Strength
target = concrete_data['Strength'] # Strength column

In [11]:
predictors.head()

,Cement,Blast Furnace Slag,Fly Ash,Water,Superplasticizer,Coarse Aggregate,Fine Aggregate,Age
0,540.0,0.0,0.0,162.0,2.5,1040.0,676.0,28
1,540.0,0.0,0.0,162.0,2.5,1055.0,676.0,28
2,332.5,142.5,0.0,228.0,0.0,932.0,594.0,270
3,332.5,142.5,0.0,228.0,0.0,932.0,594.0,365
4,198.6,132.4,0.0,192.0,0.0,978.4,825.5,360


## Normalize data

Just like the Part B, we have to normalize our data by subtracting mean value and dividing by standard deviation.

In [12]:
predictors= (predictors - predictors.mean()) / predictors.std()
predictors.head()

,Cement,Blast Furnace Slag,Fly Ash,Water,Superplasticizer,Coarse Aggregate,Fine Aggregate,Age
0,2.476712,-0.856472,-0.846733,-0.916319,-0.620147,0.862735,-1.217079,-0.279597
1,2.476712,-0.856472,-0.846733,-0.916319,-0.620147,1.055651,-1.217079,-0.279597
2,0.491187,0.795140,-0.846733,2.174405,-1.038638,-0.526262,-2.239829,3.551340
3,0.491187,0.795140,-0.846733,2.174405,-1.038638,-0.526262,-2.239829,5.055221
4,-0.790075,0.678079,-0.846733,0.488555,-1.038638,0.070492,0.647569,4.976069


And lastly, we should check target status before we move on.

In [13]:
target.head()

0    79.99
1    61.89
2    40.27
3    41.05
4    44.30
Name: Strength, dtype: float64

In [14]:
n_cols = predictors.shape[1] # number of predictors
n_cols

8

Target status also looks fine in here.

## Import Keras and make regression model

Now, we have to build our regression model.

But before we go further, we put 'sess' line so we can run multiple gpu allowed jupyter nobtebooks.(You can skip this if you just use gpu option)

In [15]:
import tensorflow as tf
import keras
from keras.models import Sequential
from keras.layers import Dense
sess = tf.compat.v1.Session(config=tf.compat.v1.ConfigProto(log_device_placement=True))


Using TensorFlow backend.


It looks fine so we are now going to build our regression model.

In [16]:
# define regression model
def regression_model():
    # create model
    model = Sequential()
    #layer 10 , relu activation
    model.add(Dense(10, activation='relu', input_shape=(n_cols,)))
    model.add(Dense(1))
    # compile model
    model.compile(optimizer='adam', loss='mean_squared_error')
    return model

And now, we have to normalize our datas so that we can get better outcome.

In [17]:
import numpy as np
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(predictors, target, test_size=0.3, random_state=42)

In [18]:
model = regression_model()

Next, we are going to train our model. However, compared to Part B, the model's epochs will be 100 instead of 50.

In [19]:
#fit the model
epochs = 100 #=> instead of 50, it is 100
model.fit(X_train, y_train, epochs=epochs, verbose=1)
loss_val = model.evaluate(X_test, y_test)
y_pred = model.predict(X_test)
loss_val

Epoch 1/100
721/721 [==============================] - 0s 492us/step - loss: 1607.0392
Epoch 2/100
721/721 [==============================] - 0s 41us/step - loss: 1591.4043
Epoch 3/100
721/721 [==============================] - 0s 39us/step - loss: 1575.9251
Epoch 4/100
721/721 [==============================] - 0s 44us/step - loss: 1560.2519
Epoch 5/100
721/721 [==============================] - 0s 39us/step - loss: 1544.3911
Epoch 6/100
721/721 [==============================] - 0s 40us/step - loss: 1528.0551
Epoch 7/100
721/721 [==============================] - 0s 40us/step - loss: 1510.9588
Epoch 8/100
721/721 [==============================] - 0s 39us/step - loss: 1493.1342
Epoch 9/100
721/721 [==============================] - 0s 46us/step - loss: 1474.3419
Epoch 10/100
721/721 [==============================] - 0s 37us/step - loss: 1454.4240
Epoch 11/100
721/721 [==============================] - 0s 39us/step - loss: 1433.0535
Epoch 12/100
721/721 [=============================

721/721 [==============================] - 0s 42us/step - loss: 169.0622
Epoch 97/100
721/721 [==============================] - 0s 43us/step - loss: 167.9404
Epoch 98/100
721/721 [==============================] - 0s 41us/step - loss: 166.9000
Epoch 99/100
721/721 [==============================] - 0s 41us/step - loss: 165.7871
Epoch 100/100
309/309 [==============================] - 0s 81us/step


171.53589998016852

## MSE compared when we increase epochs

Now we have to get MSE list. But we must print our value before we go.

In [20]:
from sklearn.metrics import mean_squared_error
mean_square_error = mean_squared_error(y_test, y_pred)
mean = np.mean(mean_square_error)
standard_deviation = np.std(mean_square_error)
print(mean, standard_deviation)

171.53590445004806 0.0


Total epochs are 100, so we have set total_mean_squared_errors as 100.

After that, I followed the same steps as Part B to see the differences.

In [21]:
total_mean_squared_errors = 100
epochs = 100
mean_squared_errors = []
for i in range(0, total_mean_squared_errors):
    X_train, X_test, y_train, y_test = train_test_split(predictors, target, test_size=0.3, random_state=i)
    model.fit(X_train, y_train, epochs=epochs, verbose=0)
    MSE = model.evaluate(X_test, y_test, verbose=0)
    print("MSE "+str(i+1)+": "+str(MSE))
    y_pred = model.predict(X_test)
    mean_square_error = mean_squared_error(y_test, y_pred)
    mean_squared_errors.append(mean_square_error)

mean_squared_errors = np.array(mean_squared_errors)
mean = np.mean(mean_squared_errors)
standard_deviation = np.std(mean_squared_errors)

print('\n')
print("Mean: "+str(mean))
print("Standard Deviation: "+str(standard_deviation))

MSE 1: 76.83672992002617
MSE 2: 63.511870202123156
MSE 3: 40.97540006051172
MSE 4: 43.32605026146355
MSE 5: 41.015358082299095
MSE 6: 43.94260343533118
MSE 7: 43.4934336591307
MSE 8: 34.4410211692736
MSE 9: 34.8099900649975
MSE 10: 36.23874733440313
MSE 11: 37.09520031333356
MSE 12: 30.501118261837266
MSE 13: 38.35746959266539
MSE 14: 39.65510135638289
MSE 15: 33.37883027703245
MSE 16: 27.474140889436296
MSE 17: 32.7634971643343
MSE 18: 32.04127380840215
MSE 19: 31.657792606785844
MSE 20: 33.05145158736837
MSE 21: 29.679085197572185
MSE 22: 29.834423065185547
MSE 23: 27.910352713081828
MSE 24: 31.627121983994172
MSE 25: 32.828759313787074
MSE 26: 35.92489532668228
MSE 27: 27.843776678190263
MSE 28: 30.00452905796878
MSE 29: 34.6071136005488
MSE 30: 30.42341327358604
MSE 31: 29.62624187531209
MSE 32: 28.447874035264295
MSE 33: 30.07278527565373
MSE 34: 29.486615770457245
MSE 35: 33.635657918491795
MSE 36: 35.83830259378674
MSE 37: 26.99429078703945
MSE 38: 30.573395750669214
MSE 39: 30.

MSE in part B Mean: 42.8598274048752

MSE in part C Mean: 31.90416229577972

In conclustion, MSE is getting much lower(almost 15%) when we set epochs to 100 instead of 50.

Judging by lots of data scientists in the internet. If your model is still improving (according to the validation loss), then more epochs are better. You can confirm this by using a hold-out test set to compare model checkpoints e.g. at epoch 100, 200, 400, 500.

Since we just doubled our epochs in a range of 100 to 500 we can assume this will lead us to get a better outcome.

But, this does not mean 1000 or 5000 epochs give us a better outcome. This may can lead to over fit action to your CNN so we have to be careful. However, this is not our case.

In general, this gives us a conclusion that more epochs in simple model that is still improving and still not in over fit zone lead us to get more smaller MSE and that means our predictions getting more accuracy.